# Rescale rat model to match Long-Evans rat skeletal parameters

### Changes include
* Global scaling to approximate average rat size.
* Rescaling of long bones to measured lengths to preserve proportions. 

In [1]:
from dm_control import mjcf
from dm_control import viewer
from dm_control import mujoco
from dm_control import suite
import numpy as np
import re

## Define useful parameters and functions

In [2]:
base_model_path = '/home/diego/code/olveczky/dm/stac/models/to_tune/rat_may17.xml'
# temp_model_path is the model used by .../dm_control/suite/rat.py for easy iteration.
temp_model_path = '/home/diego/.envs/mujoco200_3.7/lib/python3.6/site-packages/dm_control/suite/rat_temp.xml'

def view_model():
    # Load an environment from the Control Suite.
    env = suite.load(domain_name="rat", task_name="stand")
    # Launch the viewer application.
    viewer.launch(env)

def load_model(model_path):
    return mjcf.from_path(model_path)

def write_model(model, save_path):
    s = model.to_xml_string()
    s = re.sub('rat_skin.*skn', 'rat_skin_walker.skn', s)
    with open(save_path , 'w') as f:
        f.write(s)

# Save measured data (mm) and the bone-site pairs they correspond to 
bone_lengths = {'humerus': 30.0,
                'radius': 29.6,
                'femur': 36.5,
                'tibia': 42.8,
                'metatarsal': 23.4,
                'hand': 9.0}
joint_pairs = {'humerus': ['shoulder_L', 'elbow_L'],
               'radius': ['elbow_L', 'wrist_L'],
               'femur': ['hip_L', 'knee_L'],
               'tibia': ['knee_L', 'ankle_L'],
               'metatarsal': ['ankle_L', 'toe_L'],
               'hand': ['wrist_L', 'finger_L']}

base_model = load_model(base_model_path)
write_model(base_model, temp_model_path)


## View the base model

In [3]:
def get_bone_distance(physics, joint_pair):
    joint0 = physics.named.data.site_xpos[joint_pair[0]].copy()
    joint1 = physics.named.data.site_xpos[joint_pair[1]].copy()
    length = np.sqrt(np.sum((joint0 - joint1)**2))
    return length

def get_bone_ratios(bone_dict):
    n_bones = len(bone_dict.keys())
    ratio_mat = np.zeros((n_bones,n_bones))
    ratio_dict = {}
    for i, (bone0, length0) in enumerate(bone_dict.items()):
        for j, (bone1, length1) in enumerate(bone_dict.items()):
            ratio = length0/length1
            ratio_mat[i, j] = ratio
            ratio_dict[bone0 + '-' + bone1] = ratio
            
    return ratio_dict, ratio_mat

## Apply global downscaling, and long-bone length matching

In [4]:
base_model = load_model(base_model_path)
write_model(base_model, temp_model_path)
# Globally scale down the model
def scale_model(model, global_scale_ratio=.82):
    for g in model.find_all('geom'):
        if g.pos is not None and 'eye' not in g.name:
            g.pos *= global_scale_ratio
    for b in model.find_all('body'):
        if b.pos is not None and 'eye' not in g.name:
            b.pos *= global_scale_ratio
    for s in model.find_all('site'):
        if s.pos is not None and 'eye' not in g.name:
            s.pos *= global_scale_ratio
    return model
model = scale_model(base_model)
write_model(model, temp_model_path)

env = suite.load(domain_name="rat", task_name="stand")
model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
length_difference = {k: bone_lengths[k] - model_lengths[k] for k in model_lengths.keys()}
bone_ratios, bone_ratio_mat = get_bone_ratios(bone_lengths)
model_ratios, model_ratio_mat = get_bone_ratios(model_lengths)
print('Model lengths:', model_lengths)
print('Bone lengths:', bone_lengths)
ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
print('Ratio of bone_lengths to model_lengths', ratio)

# Scale particular arm and leg joints to match literature
def scale_arms_and_legs(model):
    env = suite.load(domain_name="rat", task_name="stand")
    model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
    ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
    model_name_pairs = {'humerus': ['elbow', 'humerus', 'lower_arm'],
                   'radius': ['wrist', 'radius', 'ulna', 'hand'],
                   'femur': ['knee', 'upper_leg_L0_collision', 'upper_leg_R0_collision', 'lower_leg'],
                   'tibia': ['ankle', 'foot'],
                   'metatarsal': ['toe'],
                   'hand': ['finger', 'hand_L_collision', 'hand_R_collision']}
    for i, (bone, model_id) in enumerate(model_name_pairs.items()):
        for g in model.find_all('geom'):
            if any(part in g.name for part in model_id):
                if bone == 'radius' and any(part in g.name for part in ['hand_L_collision', 'hand_R_collision']):
                    continue
                if g.pos is not None:
                    g.pos *= ratio[i]
                if g.size is not None:
                    g.size *= ratio[i]
        for b in model.find_all('body'):
            if any(part in b.name for part in model_id):
                if b.pos is not None:
                    b.pos *= ratio[i]
        for s in model.find_all('site'):
            if any(part in s.name for part in model_id):
                if s.pos is not None:
                    s.pos *= ratio[i]
    return model
model = scale_arms_and_legs(model)
write_model(model, temp_model_path)

{'humerus': 30.162181618709212, 'radius': 29.060874040537733, 'femur': 39.69160423186266, 'tibia': 42.94997217861235, 'metatarsal': 21.810933308755704, 'hand': 9.736960511371093}
{'humerus': 30.0, 'radius': 29.6, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.4, 'hand': 10.0}
[0.9946230143177504, 1.018551608554864, 0.919589941156861, 0.9965082124386793, 1.0728564279551662, 1.0270145378859987]


## Check to make sure everything worked as planned

In [5]:
env = suite.load(domain_name="rat", task_name="stand")
model_lengths = {k: get_bone_distance(env.physics, jp)*1000 for k, jp in joint_pairs.items()}
length_difference = {k: bone_lengths[k] - model_lengths[k] for k in model_lengths.keys()}
bone_ratios, bone_ratio_mat = get_bone_ratios(bone_lengths)
model_ratios, model_ratio_mat = get_bone_ratios(model_lengths)
print('Model lengths:', model_lengths)
print('Bone lengths:', bone_lengths)
ratio = [bone_lengths[k]/model_lengths[k] for k in model_lengths.keys()]
print('Ratio of bone_lengths to model_lengths', ratio)

{'humerus': 30.0, 'radius': 29.599999999999994, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.399999999999995, 'hand': 9.999999999999995}
{'humerus': 30.0, 'radius': 29.6, 'femur': 36.5, 'tibia': 42.8, 'metatarsal': 23.4, 'hand': 10.0}
[1.0, 1.0000000000000002, 1.0, 1.0, 1.0000000000000002, 1.0000000000000004]


## View the model

In [6]:
view_model()